### Import libraries and necessary datasets

In [268]:
import psycopg2
import pandas as pd
import numpy as np

In [269]:
df_wg = pd.read_csv('full_database.csv')
df_wg.shape

(39202, 5)

In [270]:
df_wg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39202 entries, 0 to 39201
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   education_attained  39202 non-null  int64  
 1   enrollment_status   39202 non-null  int64  
 2   age                 39202 non-null  float64
 3   sample_weight       39202 non-null  float64
 4   employed            39202 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 1.5 MB


In [271]:
youth = df_wg[(df_wg.age >=16) & (df_wg.age <=24)]
youth.head()

,education_attained,enrollment_status,age,sample_weight,employed
13,21,0,24.0,20.0,1
14,21,0,24.0,15.0,1
28,18,0,19.0,30.0,1
31,13,0,17.0,36.0,1
37,16,1,19.0,36.0,1


In [272]:
#add ranges for binning of age groups
youth['age_range'] = (youth.age -16)//3
youth.head()

<ipython-input-272-3603be6b57db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['age_range'] = (youth.age -16)//3


,education_attained,enrollment_status,age,sample_weight,employed,age_range
13,21,0,24.0,20.0,1,2.0
14,21,0,24.0,15.0,1,2.0
28,18,0,19.0,30.0,1,1.0
31,13,0,17.0,36.0,1,0.0
37,16,1,19.0,36.0,1,1.0


## Total Opportunity Youth
Pulling the total number of opportunity youth as defined by: "not in school, not working, between the ages of 16 and 24" from 2016 report.

In [273]:
youth

,education_attained,enrollment_status,age,sample_weight,employed,age_range
13,21,0,24.0,20.0,1,2.0
14,21,0,24.0,15.0,1,2.0
28,18,0,19.0,30.0,1,1.0
31,13,0,17.0,36.0,1,0.0
37,16,1,19.0,36.0,1,1.0
...,...,...,...,...,...,...
39162,11,0,18.0,20.0,1,0.0
39166,19,0,20.0,17.0,1,1.0
39188,19,0,19.0,6.0,1,1.0
39190,18,0,18.0,14.0,0,0.0


In [274]:
no_diploma = (youth['education_attained'] < 16)
not_enrolled = (youth['enrollment_status'] == 0)
not_employed = (youth['employed'] == 0)

youth['opportunity_youth'] = 0
youth.loc[not_enrolled & not_employed, 'opportunity_youth'] = 1

youth.head()
#youth['opportunity_youth'] = 0
#youth.loc[(youth['enrollment_status'] == 0) & (youth['employed'] == 0), 'opportunity_youth'] = 1

<ipython-input-274-d24c649821a9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['opportunity_youth'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth
13,21,0,24.0,20.0,1,2.0,0
14,21,0,24.0,15.0,1,2.0,0
28,18,0,19.0,30.0,1,1.0,0
31,13,0,17.0,36.0,1,0.0,0
37,16,1,19.0,36.0,1,1.0,0


In [275]:
youth[youth['opportunity_youth'] == 1].count()

education_attained    279
enrollment_status     279
age                   279
sample_weight         279
employed              279
age_range             279
opportunity_youth     279
dtype: int64

In [276]:
youth

,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth
13,21,0,24.0,20.0,1,2.0,0
14,21,0,24.0,15.0,1,2.0,0
28,18,0,19.0,30.0,1,1.0,0
31,13,0,17.0,36.0,1,0.0,0
37,16,1,19.0,36.0,1,1.0,0
...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,1,0.0,0
39166,19,0,20.0,17.0,1,1.0,0
39188,19,0,19.0,6.0,1,1.0,0
39190,18,0,18.0,14.0,0,0.0,1


## Total Working Without a Diploma (and also out of school) 



In [277]:
youth['working_without_diploma'] = 0
youth.loc[no_diploma & (youth['employed'] == 1) & not_enrolled,'working_without_diploma'] = 1
youth

<ipython-input-277-95c571cfafff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['working_without_diploma'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma
13,21,0,24.0,20.0,1,2.0,0,0
14,21,0,24.0,15.0,1,2.0,0,0
28,18,0,19.0,30.0,1,1.0,0,0
31,13,0,17.0,36.0,1,0.0,0,1
37,16,1,19.0,36.0,1,1.0,0,0
...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,1,0.0,0,1
39166,19,0,20.0,17.0,1,1.0,0,0
39188,19,0,19.0,6.0,1,1.0,0,0
39190,18,0,18.0,14.0,0,0.0,1,0


## Opportunity Youth No Diploma 
I ALSO TRIED TO DO IT FROM LARGER DATA SET INSTEAD OF FROM SUBSET, results were the same

In [278]:
youth['diploma'] = 1
youth.loc[no_diploma,'diploma'] = 0
youth

<ipython-input-278-1fabab18d87f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['diploma'] = 1
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma,diploma
13,21,0,24.0,20.0,1,2.0,0,0,1
14,21,0,24.0,15.0,1,2.0,0,0,1
28,18,0,19.0,30.0,1,1.0,0,0,1
31,13,0,17.0,36.0,1,0.0,0,1,0
37,16,1,19.0,36.0,1,1.0,0,0,1
...,...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,1,0.0,0,1,0
39166,19,0,20.0,17.0,1,1.0,0,0,1
39188,19,0,19.0,6.0,1,1.0,0,0,1
39190,18,0,18.0,14.0,0,0.0,1,0,1


## Opportunity Youth High School Diploma or GED 

In [279]:
youth['diploma_GED'] = 0
youth.loc[youth['education_attained'].isin([16,17]), 'diploma_GED'] = 1
youth

<ipython-input-279-1cdd0bcaa8de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['diploma_GED'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma,diploma,diploma_GED
13,21,0,24.0,20.0,1,2.0,0,0,1,0
14,21,0,24.0,15.0,1,2.0,0,0,1,0
28,18,0,19.0,30.0,1,1.0,0,0,1,0
31,13,0,17.0,36.0,1,0.0,0,1,0,0
37,16,1,19.0,36.0,1,1.0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,1,0.0,0,1,0,0
39166,19,0,20.0,17.0,1,1.0,0,0,1,0
39188,19,0,19.0,6.0,1,1.0,0,0,1,0
39190,18,0,18.0,14.0,0,0.0,1,0,1,0


## Opportunity Youth Some College, No Degree 

In [280]:
youth['some_college_no_degree'] = 0
youth.loc[youth['education_attained'].isin([18,19]), 'some_college_no_degree'] = 1
youth

<ipython-input-280-f63010292aca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['some_college_no_degree'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma,diploma,diploma_GED,some_college_no_degree
13,21,0,24.0,20.0,1,2.0,0,0,1,0,0
14,21,0,24.0,15.0,1,2.0,0,0,1,0,0
28,18,0,19.0,30.0,1,1.0,0,0,1,0,1
31,13,0,17.0,36.0,1,0.0,0,1,0,0,0
37,16,1,19.0,36.0,1,1.0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,1,0.0,0,1,0,0,0
39166,19,0,20.0,17.0,1,1.0,0,0,1,0,1
39188,19,0,19.0,6.0,1,1.0,0,0,1,0,1
39190,18,0,18.0,14.0,0,0.0,1,0,1,0,1


## Opportunity Youth Degree(Associate or Higher)

In [281]:
youth['assoc_or_higher'] = 0
youth.loc[youth['education_attained'] >= 20, 'assoc_or_higher'] = 1
youth

<ipython-input-281-76895dc147ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youth['assoc_or_higher'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,education_attained,enrollment_status,age,sample_weight,employed,age_range,opportunity_youth,working_without_diploma,diploma,diploma_GED,some_college_no_degree,assoc_or_higher
13,21,0,24.0,20.0,1,2.0,0,0,1,0,0,1
14,21,0,24.0,15.0,1,2.0,0,0,1,0,0,1
28,18,0,19.0,30.0,1,1.0,0,0,1,0,1,0
31,13,0,17.0,36.0,1,0.0,0,1,0,0,0,0
37,16,1,19.0,36.0,1,1.0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
39162,11,0,18.0,20.0,1,0.0,0,1,0,0,0,0
39166,19,0,20.0,17.0,1,1.0,0,0,1,0,1,0
39188,19,0,19.0,6.0,1,1.0,0,0,1,0,1,0
39190,18,0,18.0,14.0,0,0.0,1,0,1,0,1,0


# RENAME SAMPLE WEIGHT COLUMNS

# Merged Table-- Totals of OY no degree, High School/GED, some college no degree, college degree
ADD TOTAL OY COLUMN

In [282]:
total_population = pd.DataFrame()
total_population['Total Population'] = youth.groupby('age_range').sum()['sample_weight']
total_population['Opportunity Youth'] = youth[youth['opportunity_youth'] == 1].groupby('age_range').sum()['sample_weight']
total_population['Working Without Diploma'] = youth[youth['working_without_diploma'] == 1].groupby('age_range').sum()['sample_weight']
total_population['Not an Opportunity Youth'] = youth[youth['opportunity_youth'] == 0].groupby('age_range').sum()['sample_weight']

total_population

,Total Population,Opportunity Youth,Working Without Diploma,Not an Opportunity Youth
age_range,,,,
0.0,33791.0,2203.0,23387.0,31588.0
1.0,29915.0,2324.0,3300.0,27591.0
2.0,36277.0,2134.0,2670.0,34143.0


In [283]:
oy = youth[youth['opportunity_youth'] == 1]

In [284]:
opportunity_youth = pd.DataFrame()
opportunity_youth['Opportunity Youth'] =oy.groupby('age_range').sum()['sample_weight']
opportunity_youth['No diploma'] = oy[oy.diploma == 0].groupby('age_range').sum()['sample_weight']
opportunity_youth['HS diploma or GED'] = oy[oy['diploma_GED'] == 1].groupby('age_range').sum()['sample_weight']
opportunity_youth['Some College, no degree'] = oy[oy['some_college_no_degree'] == 1].groupby('age_range').sum()['sample_weight']
opportunity_youth['Degree (Associate or higher)'] = oy[oy['assoc_or_higher'] == 1].groupby('age_range').sum()['sample_weight']
opportunity_youth.fillna(0, inplace = True)
opportunity_youth

,Opportunity Youth,No diploma,HS diploma or GED,"Some College, no degree",Degree (Associate or higher)
age_range,,,,,
0.0,2203.0,1671.0,280.0,252.0,0.0
1.0,2324.0,540.0,832.0,763.0,189.0
2.0,2134.0,296.0,742.0,595.0,501.0


In [285]:
total_population

,Total Population,Opportunity Youth,Working Without Diploma,Not an Opportunity Youth
age_range,,,,
0.0,33791.0,2203.0,23387.0,31588.0
1.0,29915.0,2324.0,3300.0,27591.0
2.0,36277.0,2134.0,2670.0,34143.0


In [286]:
for col in opportunity_youth.columns:
    opportunity_youth[col + '%'] = (100*opportunity_youth[col]/opportunity_youth['Opportunity Youth']).round(2)
    opportunity_youth[col + '%'] = opportunity_youth[col + '%'].astype(str) + '%'
opportunity_youth

,Opportunity Youth,No diploma,HS diploma or GED,"Some College, no degree",Degree (Associate or higher),Opportunity Youth%,No diploma%,HS diploma or GED%,"Some College, no degree%",Degree (Associate or higher)%
age_range,,,,,,,,,,
0.0,2203.0,1671.0,280.0,252.0,0.0,100.0%,75.85%,12.71%,11.44%,0.0%
1.0,2324.0,540.0,832.0,763.0,189.0,100.0%,23.24%,35.8%,32.83%,8.13%
2.0,2134.0,296.0,742.0,595.0,501.0,100.0%,13.87%,34.77%,27.88%,23.48%


In [287]:
col1 = opportunity_youth.columns
col1

Index(['Opportunity Youth', 'No diploma', 'HS diploma or GED',
       'Some College, no degree', 'Degree (Associate or higher)',
       'Opportunity Youth%', 'No diploma%', 'HS diploma or GED%',
       'Some College, no degree%', 'Degree (Associate or higher)%'],
      dtype='object')

In [288]:
col1 = ['Opportunity Youth','Opportunity Youth%',
        'No diploma','No diploma%', 
        'HS diploma or GED', 'HS diploma or GED%',
       'Some College, no degree', 'Some College, no degree%',
       'Degree (Associate or higher)','Degree (Associate or higher)%']
opportunity_youth = opportunity_youth[col1]
opportunity_youth

,Opportunity Youth,Opportunity Youth%,No diploma,No diploma%,HS diploma or GED,HS diploma or GED%,"Some College, no degree","Some College, no degree%",Degree (Associate or higher),Degree (Associate or higher)%
age_range,,,,,,,,,,
0.0,2203.0,100.0%,1671.0,75.85%,280.0,12.71%,252.0,11.44%,0.0,0.0%
1.0,2324.0,100.0%,540.0,23.24%,832.0,35.8%,763.0,32.83%,189.0,8.13%
2.0,2134.0,100.0%,296.0,13.87%,742.0,34.77%,595.0,27.88%,501.0,23.48%


In [289]:
for col in total_population.columns:
    total_population[col + '%'] = (100*total_population[col]/total_population['Total Population']).round(2)
    total_population[col + '%'] = total_population[col + '%'].astype(str) + '%'
total_population

,Total Population,Opportunity Youth,Working Without Diploma,Not an Opportunity Youth,Total Population%,Opportunity Youth%,Working Without Diploma%,Not an Opportunity Youth%
age_range,,,,,,,,
0.0,33791.0,2203.0,23387.0,31588.0,100.0%,6.52%,69.21%,93.48%
1.0,29915.0,2324.0,3300.0,27591.0,100.0%,7.77%,11.03%,92.23%
2.0,36277.0,2134.0,2670.0,34143.0,100.0%,5.88%,7.36%,94.12%


In [290]:
col2 = total_population.columns
col2

Index(['Total Population', 'Opportunity Youth', 'Working Without Diploma',
       'Not an Opportunity Youth', 'Total Population%', 'Opportunity Youth%',
       'Working Without Diploma%', 'Not an Opportunity Youth%'],
      dtype='object')

In [291]:
col2 = ['Total Population', 'Total Population%',
        'Opportunity Youth','Opportunity Youth%', 
        'Working Without Diploma', 'Working Without Diploma%',
       'Not an Opportunity Youth','Not an Opportunity Youth%']
total_population = total_population[col2]
total_population

,Total Population,Total Population%,Opportunity Youth,Opportunity Youth%,Working Without Diploma,Working Without Diploma%,Not an Opportunity Youth,Not an Opportunity Youth%
age_range,,,,,,,,
0.0,33791.0,100.0%,2203.0,6.52%,23387.0,69.21%,31588.0,93.48%
1.0,29915.0,100.0%,2324.0,7.77%,3300.0,11.03%,27591.0,92.23%
2.0,36277.0,100.0%,2134.0,5.88%,2670.0,7.36%,34143.0,94.12%


In [292]:
total_population.reset_index(inplace = True)

In [293]:
ages = ['16-18','19-21','22-24']

In [294]:
total_population.reset_index(inplace = True)
total_population.loc[:,'ages'] = ages
total_population.set_index('ages',inplace = True)
total_population.drop(columns = ['age_range', 'index'], inplace = True)

total_population

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Total Population,Total Population%,Opportunity Youth,Opportunity Youth%,Working Without Diploma,Working Without Diploma%,Not an Opportunity Youth,Not an Opportunity Youth%
ages,,,,,,,,
16-18,33791.0,100.0%,2203.0,6.52%,23387.0,69.21%,31588.0,93.48%
19-21,29915.0,100.0%,2324.0,7.77%,3300.0,11.03%,27591.0,92.23%
22-24,36277.0,100.0%,2134.0,5.88%,2670.0,7.36%,34143.0,94.12%


In [295]:
opportunity_youth.reset_index(inplace = True)
opportunity_youth.loc[:,'ages'] = ages
opportunity_youth.set_index('ages',inplace = True)
opportunity_youth.drop(columns = ['age_range'], inplace = True)
opportunity_youth

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Opportunity Youth,Opportunity Youth%,No diploma,No diploma%,HS diploma or GED,HS diploma or GED%,"Some College, no degree","Some College, no degree%",Degree (Associate or higher),Degree (Associate or higher)%
ages,,,,,,,,,,
16-18,2203.0,100.0%,1671.0,75.85%,280.0,12.71%,252.0,11.44%,0.0,0.0%
19-21,2324.0,100.0%,540.0,23.24%,832.0,35.8%,763.0,32.83%,189.0,8.13%
22-24,2134.0,100.0%,296.0,13.87%,742.0,34.77%,595.0,27.88%,501.0,23.48%


In [297]:
opportunity_youth.to_csv('opportunity_youth.csv', index = False)
total_population.to_csv('total_population.csv', index = False)